### Build and Train 3DGAN


In [ ]:
import os
import torch
from torch import optim
from torch import nn
from utils import utils3D
from torch.utils import data
from torch.autograd import Variable
from models.threed.gan import GAN
import matplotlib
import pickle
matplotlib.use('agg')

#### Variables to define size of latent feature, learning rates of G and D ... beta parameter for adam and batch size

In [ ]:
Z_LATENT_SPACE = 200
G_LR = 0.0025
D_LR = 0.001
EPOCHS = 1
BETA = (0.5, 0.5) 
BSIZE = 32
CUBE_LEN = 64

#### Define model

In [ ]:
gan3D = GAN(epochs=EPOCHS, sample=8, 
            batch=BSIZE, betas=BETA,
            g_lr=G_LR, d_lr=D_LR, cube_len=CUBE_LEN, latent_v=Z_LATENT_SPACE)

In [ ]:
gan3D.train()
print("Training finished!")

### Load and plot trained nets

In [ ]:
import torch
from models.threed.generator import _G
from models.threed.discriminator import _D
from utils import utils3D
import skimage.measure as sk
import visdom
import trimesh
import numpy as np

In [ ]:
vis = visdom.Visdom()
D = _D().cuda()
G = _G(z_latent_space=200).cuda()
G.load_state_dict(torch.load('/tmp/faces/GAN3D/GAN3D_G.pkl'))
D.load_state_dict(torch.load('/tmp/faces/GAN3D/GAN3D_D.pkl'))

In [ ]:
Z_LATENT_SPACE = 200
BATCH_SIZE = 32

Z = utils3D.var_or_cuda(torch.randn(BATCH_SIZE, Z_LATENT_SPACE))   
fake = G(Z)
samples = fake.cpu().data[:4].squeeze().numpy()
for s, sample in enumerate(samples):
    utils3D.plotVoxelVisdom(str(s), sample, 0.0001, vis, "3D vessels")


#### Save to stl generated vessels

In [ ]:
samples[0].shape, np.max(samples[0]),  np.min(samples[0])

In [ ]:
v, f = sk.marching_cubes_classic(samples[0], level=0.01)
sample_mesh = trimesh.Trimesh(v, f)
sample_mesh.export('/tmp/test.stl')

In [ ]:
v, f = sk.marching_cubes_classic(samples[0], level=0.5)